## TSV to CSV:

In [ ]:
import pandas as pd
import numpy as np
from random import sample
from cmath import exp, sqrt
import heapq
import random
from collections import deque 
import statistics as stats

In [ ]:
file_name1 = 'out.soc-sign-bitcoinalpha'
def tsv_to_csv(file_name):
	with open(file_name, "rt") as file:
		x = file.read()
	with open(file_name, "wt") as file:
		x = x.replace('% asym positive','from\tto\tweight\ttime')
		x = x.replace('% sym positive','from\tto\tweight\ttime')
		x = x.replace('% asym signed','from\tto\tweight\ttime')
		x = x.replace('% asym negative','from\tto\ttime')
		x = x.replace('  ', ' ')
		x = x.replace(' ', '\t')
		x = x.replace(' ', '')
		x = x.replace('% 82927 167 167', '')

		file.write(x)

	tsv_file = file_name

	# reading given tsv file
	csv_table=pd.read_table(tsv_file,sep='\t')

	# converting tsv file into csv
	csv_table.to_csv(file_name + '.csv',index=False)
	return csv_table

# Static Graph Properties

In [ ]:
def df_to_adj_list(df):
  g = dict()
  #e = df[['from','to','weight','time']].iloc() # edges
  e = df[['from','to','time']].iloc() # edges
  vertices = pd.concat([df['from'],df['to']]).unique()
  for i in vertices:
    g[i] = []
  for i in range(len(df[['from']])):
    if (not e[i][1] in g[ e[i][0] ]):
      g[ e[i][0] ].append(
      (e[i][1],), # to
        ) # g[from][i] = [ (to(j),) ] - храним в кортеже, чтобы можно было добавить другие характеристики, если понадобяться
        # Но проблема в проверке, добавлено ребро или нет, если там различные веса
        # Так что оставляем только номер узла, в которое можем попасть
    if (not e[i][0] in g[ e[i][1] ]):
      g[ e[i][1] ].append((e[i][0],))
# # Убираем вершины, из которых нет ребер
#   for u in list(g):
#     if (g[u] == []):
#       g.pop(u)
  return g

# Вычисление числа компонент слабой связи (к.с.с.) и нахождение максимальной к.с.с.
def comp_counter(g):
  #visited = [False for _ in range(nvmax+1)]
  visited = dict()
  for u in g:
    visited[u] = False
  wccs = [] # weakly connected components
  temp_dict = dict()
  def dfs(v):
    stack = []
    stack.append(v)
    while (stack):
      v = stack.pop()
      if (v in g):
        visited[v] = True
        temp_dict[v] = g[v]
        for e in g[v]:
          if not visited[e[0]]:  # посещён ли текущий сосед?
            stack.append(e[0])
    return temp_dict

  nc = 0 
  for v in g:
    if not visited[v]:
        dfs(v)
        wccs.append(temp_dict) # Сохраняем все к.с.с.
        temp_dict = dict()
        nc += 1 # Подсчет к.с.с.

  # Находим максимальную к.с.с.
  len_of_wccs = [len(wccs[i]) for i in range(len(wccs))]
  max_len = max(len_of_wccs)
  index_max = len_of_wccs.index(max_len)
  max_wcc = wccs[index_max]
  
  return nc, max_wcc

In [ ]:
def rad_diam_perc(graph):
  # Воспользуемся алгоритмом Дейкстры
  def dijkstras(g, root): # На вход подается граф в виде списков смежности
    dist = dict() 
    for u in g:
      dist[u] = np.inf
    # Изначальное расстояние до других вершин = 0
    dist[root] = 0 # Расстояние до корневой вершины = 0
    visited = dict()
    for u in g:
      visited[u] = False
    pq = [(0, root)] # Очередь с приоритетами (dist[node],node), первый элемент - корневой узел

    while pq:
      _, u = heapq.heappop(pq) # Рассматриваем текущий узел
      # Если узел посещали - пропускаем
      if visited[u]:
          continue
      visited[u] = True
      if (u in g):
        for e in g[u]:
          # Вытаскиваем номер вершины, которую мы хотим посетить
          v = e[0] 
          if (v in g):
            if (dist[u] + 1 < dist[v]):
              dist[v] = dist[u] + 1
              heapq.heappush(pq, (dist[v], v))
    
    for v in list(dist):
      if (dist[v] == np.inf):
        dist.pop(v)
    return list(dist.values())

  # Создаем "снежный ком" из ~500 вершин
  def snowball_sample(u):
    snowball = dict()
    q = deque()
    k = 0
    snowball[u] = graph[u]
    k+=1
    while (k<500):
      for e in graph[u]:
        q.append(e[0])
        snowball[ e[0] ] = graph[ e[0] ]
        k+=1
      u = q.popleft()
    return snowball    

  extr = []
  perc90th = []
  rad = []
  diam = []
  perc90th_of_dist = []
  if (len(graph) > 600):
    rc_for_snowball = []
    rc_for_wcc = []
    # Начальные вершины для 10-ти "снежных комов"
    for _ in range(10):
      rc_for_snowball.append(random.choice(list(graph.keys())))
    # 500 случайных вершин для построения подграфа
    for _ in range(500):
      rc_for_wcc.append(random.choice(list(graph.keys())))
    
    for ru in rc_for_snowball:
      snowball = snowball_sample(ru) 
      for u in snowball:
        distu = dijkstras(snowball,u)
        perc90th.append(np.percentile(distu,90))
        extr.append(max(distu))
      rad.append(min(extr))
      diam.append(max(extr))
      perc90th_of_dist.append(np.percentile(perc90th, 90))
      extr = []
      perc90th = []
    for ru in rc_for_wcc:
      distu = dijkstras(graph,ru)
      perc90th.append(np.percentile(distu,90))
      extr.append(max(distu))
    rad.append(min(extr))
    diam.append(max(extr))
    perc90th_of_dist.append(np.percentile(perc90th, 90))

  else:
    for u in graph:
      distu = dijkstras(graph,u)
      perc90th.append(np.percentile(distu,90))
      extr.append(max(distu))
    rad.append(min(extr))
    diam.append(max(extr))
    perc90th_of_dist.append(np.percentile(perc90th, 90))

  # Выбираем наиболее часто встречающиеся результаты
  rad_med = stats.mode(rad)
  diam_med = stats.mode(diam)
  perc90th_of_dist_med = stats.mode(perc90th_of_dist)
  

  return rad_med, diam_med, perc90th_of_dist_med

def acnc_assort(g): # average clustering network coefficient
  C = 0
  R1 = 0
  R2 = 0
  R3 = 0
  Re = 0

  for u in g:
    degu = len(g[u])
    
    R1 += degu
    R2 += degu**2
    R3 += degu**3
    Lu=0
    if (degu < 2):
      for e in g[u]:
        Re += degu * len(g[ e[0] ])
    else:
      for e in g[u]:
        Re += degu * len(g[ e[0] ])
        for e2 in g[u]:
          for e3 in g[ e[0] ]:
            if (e2[0] == e3[0]):
              Lu+=1
      Lu = Lu*0.5
      C += 2*Lu/((degu)*(degu-1))

  C = C*(1/len(g))
  r = (Re*R1-R2**2)/(R3*R1-R2**2)
  return C, r

In [ ]:
datasets = ['out_radoslaw_email_email.csv','out_opsahl-ucsocial.csv','email-Eu-core-temporal.csv','out.soc-sign-bitcoinalpha.csv','out_soc-sign-bitcoinotc.csv']

,from,to,weight,time
0,1,2,10,1407470400
1,3,2,10,1376539200
2,4,2,10,1369713600
3,5,2,10,1350014400
4,6,2,10,1347854400
...,...,...,...,...
24181,885,1251,10,1364270400
24182,1251,885,10,1364270400
24183,885,1250,10,1364270400
24184,1250,885,10,1364270400


In [ ]:
for dataset in datasets:
  csv_table = pd.read_csv(dataset)
  nvmax = pd.concat([csv_table['from'],csv_table['to']]).nunique() # number of vertices
  t0 = csv_table['time'].min()
  tmax = csv_table['time'].max()
  timestamps = np.sort(csv_table['time'].unique())
  
  g = df_to_adj_list(csv_table)
  print('Характеристики статического графа для дата-сета: ', dataset,'\n')
  # Number of vertices
  nv = len(g)
  print('Количество вершин: ', nv)

  # Number of edges
  ne = 0
  for u in g:
    ne += len(g[u])
  ne = int(ne*0.5)

  print('Количество ребер: ', ne)

  # Graph density
  gd = ne / (nv*(nv-1)/2)
  print('Плотность графа: ', gd)

  # Number of weakly connected components

  nc, max_wcc = comp_counter(g)
  print('Количество компонент слабой связности: ', nc)

  # Fraction of vertices in max weak connectivity component
  fvmwcc = len(max_wcc)/nv
  print('Доля вершин в максимальной компоненте слабой связности: ', fvmwcc)

  # Radius 
  rad, diam, perc90th_of_dist = rad_diam_perc(max_wcc)
  print('Радиус: ', rad)

  # Diameter
  print('Диаметр: ', diam)

  # 90th percentile of distance (geodesic) between graph vertices
  print('90 процентиль расстояния (геодезического) между вершинами: ', perc90th_of_dist)

  # Average clustering network coefficient
  # Clustering coefficient is a property of a node in a network. Roughly speaking it tells how well connected the neighborhood of the node is. 
  # If the neighborhood is fully connected, the clustering coefficient is 1 and a value close to 0 means that there are hardly any connections in the neighborhood.

  # The overall probability that two neighbours of a randomly selected node are linked to each other
  acn_coef, assort_coef = acnc_assort(g)
  print('Средний кластерный коэффициент: ', acn_coef)

  # Vertex degree assortative coefficient (Pearson's coefficient)
  # Положительные значения r обозначают корреляцию между узлами схожих степеней, а отрицательные значения обозначают отношения между узлами разных степеней.
  print('Коэффициент ассортативности по степени вершины (коэф.корреляции Пирсона): ', assort_coef)

  print('\n', '------------------------------------','\n')